# Selenium - 동적 웹페이지 스크래핑

In [4]:
# Seleium 드라이버 생성
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 화면없이 실행

# 드라이버 서비스 생성
service = Service(ChromeDriverManager().install())

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

## 뉴스 서비스에 접속

In [5]:
# 뉴스 사이트
url = "https://news.daum.net/"

driver.get(url)

In [14]:
# 페이지 소스 가져오기
from bs4 import BeautifulSoup
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')

# 태그 검색
print('title 태그 요소: ', soup.title)
print('title 태그 이름: ', soup.title.name)
print('title 태그 문자열: ', soup.title.text)

title 태그 요소:  <title>다음뉴스 | 경제</title>
title 태그 이름:  title
title 태그 문자열:  다음뉴스 | 경제


## 경제 카테고리를 선택

In [15]:
from selenium.webdriver.common.by import By

# 경제 뉴스 클릭
economy = driver.find_element(By.CSS_SELECTOR, '#gnbContent > div > ul > li:nth-child(4) > a > span')
economy

<selenium.webdriver.remote.webelement.WebElement (session="43c1bdf9446760cee7ef84b9d9ecaa83", element="f.C08EBC2C93FACCFA2F8B071E74A2D74F.d.6EDFEFCC39C5F1F818A44FC803C5E7D4.e.2473")>

In [16]:
economy.click()

driver.implicitly_wait(5)

In [17]:
# 페이지 소스 가져오기

soup = BeautifulSoup(driver.page_source, 'html.parser')

# 실시간 경제 뉴스 검색
news = soup.select('#timeline > ul > li')
len(news)

20

In [18]:
news[0]

<li>
<em class="txt_time">06:22</em>
<strong class="tit_g">
<span class="info_cp">연합뉴스</span>
<a class="link_txt" data-tiara-custom="contentUniqueKey=hamny-20240824062204933" data-tiara-id="20240824062204933" data-tiara-layer="article" data-tiara-ordnum="1" data-tiara-type="harmony" href="https://v.daum.net/v/20240824062204933">뉴욕증시, 파월이 쏘아 올린 피벗 신호탄…나스닥 1.47%↑ 마감</a>
</strong>
</li>

In [19]:
news[0].select("a.link_txt")[0].text.strip()

'뉴욕증시, 파월이 쏘아 올린 피벗 신호탄…나스닥 1.47%↑ 마감'

In [20]:
news[0].select_one("a.link_txt").text.strip()

'뉴욕증시, 파월이 쏘아 올린 피벗 신호탄…나스닥 1.47%↑ 마감'

In [21]:
# 데이터 추출하여 정리
data = {'title':[], 'agency':[], 'category':[], 'link':[]}

for item in news:
    title = item.select_one('strong > a').text
    agency = item.select_one('span.info_cp').text
    category = '경제'
    link = item.select_one('strong > a')['href']
    
    data['title'].append(title)
    data['agency'].append(agency)
    data['category'].append(category)
    data['link'].append(link)


# 데이터 프레임 생성
import pandas as pd
df = pd.DataFrame(data)
df.head()

,title,agency,category,link
0,"뉴욕증시, 파월이 쏘아 올린 피벗 신호탄…나스닥 1.47%↑ 마감",연합뉴스,경제,https://v.daum.net/v/20240824062204933
1,"""내가 살 집, 호텔처럼 예약""…MZ, 따로 또 같이 산다[르포]",뉴스1,경제,https://v.daum.net/v/20240824062004912
2,"올해 수도권 1순위 청약자 10명 중 8명, '분상제 아파트'로",데일리안,경제,https://v.daum.net/v/20240824061643892
3,"나스닥 1.5% 랠리…파월 비둘기 등판 ""금리인하 시간이 됐다""[뉴욕마감]",뉴스1,경제,https://v.daum.net/v/20240824061635889
4,"‘빅컷’ 가능성 남긴 파월…환호한 뉴욕증시, 나스닥 1.5%↑[월스트리트in]",이데일리,경제,https://v.daum.net/v/20240824061250862


In [22]:
len(df)

20

## 페이지 이동하면서 반복적으로 수집

In [23]:
# 페이지네이션 배열
#timeline > div > div > a

page_numbers = driver.find_elements(By.CSS_SELECTOR, '#timeline > div > div > a')
len(page_numbers)

4

In [24]:
page_numbers[0]

<selenium.webdriver.remote.webelement.WebElement (session="43c1bdf9446760cee7ef84b9d9ecaa83", element="f.C08EBC2C93FACCFA2F8B071E74A2D74F.d.34CA8608590C899A060EB713EACF15EA.e.4000")>

In [25]:
for p in page_numbers:
    print(p.text)

2
3
4
5


In [26]:
current_page = 1

In [27]:
for p in page_numbers:
    print(p.text)
    if (p.text != '다음') and (int(p.text) == current_page + 1):
        p.click()
        driver.implicitly_wait(5)
        current_page = current_page + 1
        print(p.text, current_page)
        break

2
2 2


In [28]:
current_page

2

In [29]:
# 페이지 소스 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 실시간 경제 뉴스 검색
news = soup.select('#timeline > ul > li')
len(news)

20

In [30]:
# 뉴스 추출하여 기존 데이터에 추가
for item in news:
    title = item.select_one('strong > a').text
    agency = item.select_one('span.info_cp').text
    category = '경제'
    link = item.select_one('strong > a')['href']
    
    data['title'].append(title)
    data['agency'].append(agency)
    data['category'].append(category)
    data['link'].append(link)


# 데이터 프레임 생성
df = pd.DataFrame(data)
print(len(df))

df.head()

40


,title,agency,category,link
0,"뉴욕증시, 파월이 쏘아 올린 피벗 신호탄…나스닥 1.47%↑ 마감",연합뉴스,경제,https://v.daum.net/v/20240824062204933
1,"""내가 살 집, 호텔처럼 예약""…MZ, 따로 또 같이 산다[르포]",뉴스1,경제,https://v.daum.net/v/20240824062004912
2,"올해 수도권 1순위 청약자 10명 중 8명, '분상제 아파트'로",데일리안,경제,https://v.daum.net/v/20240824061643892
3,"나스닥 1.5% 랠리…파월 비둘기 등판 ""금리인하 시간이 됐다""[뉴욕마감]",뉴스1,경제,https://v.daum.net/v/20240824061635889
4,"‘빅컷’ 가능성 남긴 파월…환호한 뉴욕증시, 나스닥 1.5%↑[월스트리트in]",이데일리,경제,https://v.daum.net/v/20240824061250862


In [31]:
# 드라이버 종료
driver.close()

In [41]:
# 페이지를 이동하면서 데이터를 수집하여 정리하는 함수
def get_news(driver, category):

    data = {'title':[], 'agency':[], 'category':[], 'link':[]}

    current_page = 1
    
    while current_page < 5:
        try:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            news = soup.select('#timeline > ul > li')
            
            for item in news:
                title = item.select_one('strong > a').text
                agency = item.select_one('span.info_cp').text
                link = item.select_one('strong > a')['href']
                
                data['title'].append(title)
                data['agency'].append(agency)
                data['category'].append(category)
                data['link'].append(link)
            
            page_numbers = driver.find_elements(By.CSS_SELECTOR, '#timeline > div > div > a')
            driver.implicitly_wait(5)
            
            for p in page_numbers:
                if (p.text != '다음') and (int(p.text) == current_page + 1):
                    p.click()
                    driver.implicitly_wait(5)
                    current_page = current_page + 1
                    print(p.text, current_page)
                    break        

        except:
            pass
    
    df = pd.DataFrame(data)
    
    driver.close()

    return df

In [42]:
# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

In [43]:
# 뉴스 사이트 접속
url = "https://news.daum.net/"
driver.get(url)

In [44]:
# 뉴스 카테고리 메뉴 배열
category_tabs = driver.find_elements(By.CSS_SELECTOR, '#gnbContent > div > ul > li')
category_tabs

[<selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.321")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.324")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.327")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.330")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.333")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649

In [45]:
category_tabs[3].text

'경제'

In [46]:
category_dict = {item.text:item for item in category_tabs}
category_dict

{'홈': <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.321")>,
 '사회': <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.324")>,
 '정치': <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.327")>,
 '경제': <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.330")>,
 '국제': <selenium.webdriver.remote.webelement.WebElement (session="8f3106e3a37cafe1d649e9cb322f3393", element="f.EC89B7DF0CC08CF5D782040541C00DFF.d.EB49782AF003B56E21734F777EB83AC4.e.333")>,
 '문화': <selenium.webdriver.remote.webelement.WebEle

In [47]:
# 경제 뉴스 클릭
category_dict['경제'].click()
driver.implicitly_wait(5)

In [48]:
# 데이터 수집
df_economy = get_news(driver, '경제')
print(len(df_economy))

df_economy.head()

2 2
3 3
4 4
5 5
140


,title,agency,category,link
0,"뉴욕증시, 파월이 쏘아 올린 피벗 신호탄…나스닥 1.47%↑ 마감",연합뉴스,경제,https://v.daum.net/v/20240824062204933
1,"""내가 살 집, 호텔처럼 예약""…MZ, 따로 또 같이 산다[르포]",뉴스1,경제,https://v.daum.net/v/20240824062004912
2,"올해 수도권 1순위 청약자 10명 중 8명, '분상제 아파트'로",데일리안,경제,https://v.daum.net/v/20240824061643892
3,"나스닥 1.5% 랠리…파월 비둘기 등판 ""금리인하 시간이 됐다""[뉴욕마감]",뉴스1,경제,https://v.daum.net/v/20240824061635889
4,"‘빅컷’ 가능성 남긴 파월…환호한 뉴욕증시, 나스닥 1.5%↑[월스트리트in]",이데일리,경제,https://v.daum.net/v/20240824061250862


In [49]:
# 페이지 이동할 때 동적으로 기사 배열이 바뀌면서 일부 뉴스가 중복 수집되는 현상이 있습니다. 중복을 제거합니다. 
df_economy = df_economy.drop_duplicates()
len(df_economy)

80

In [50]:
# 데이터 저장
df_economy.to_csv('daum_economy_news.csv', index=False)